# Assignment 5

You have to get all the votes for all the departments and provinces and districts available in this [link](https://resultadoshistorico.onpe.gob.pe/EG2021/EleccionesPresidenciales/RePres/T). General a final table with the columns of the department, province, district, political_organization, total_votes.
The code must run and do not fail. If the code fails, it will be graded as zero. Check this.

In [1]:
from IPython.display import display, HTML

display(HTML(data="""
<style>
    div#notebook-container    { width: 95%; }
    div#menubar-container     { width: 65%; }
    div#maintoolbar-container { width: 99%; }a
</style>
"""))

In [2]:
!pip install selenium
!pip install webdriver-manager

In [3]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.chrome.service import Service
import time 

# Configure the Chrome driver
#It sets up the Chrome driver using the executable path of the ChromeDriver and some options for the Chrome browser.

service = Service(executable_path= r"C:\Users\Marco\Downloads\chromedriver-win64\chromedriver-win64\chromedriver.exe")
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=service, options=options)

# Navigate to a Website:
#This line opens the specified website ("https://resultadoshistorico.onpe.gob.pe/EG2021/") using the Chrome driver.
driver.get("https://resultadoshistorico.onpe.gob.pe/EG2021/")

# Click on "resumen_general":
resumen_general = driver.find_element(By.XPATH, "/html/body/onpe-root/onpe-home-onpe/div[1]/div/div/div/div[2]/div[1]/div/div/a/div[1]/img")
resumen_general.click()
#It finds an HTML element on the page using XPath, and then performs a click action on it. In this case, it seems to be clicking on an image element within the webpage structure.

In [4]:
# Add 5 seconds to wait for the page to load using "time.sleep" function 
# Click on the "presidencial" Element:
# these lines of code navigate to a specific element on the webpage, identified as "presidencial," and then simulate a click on that element. This is often done in web automation to interact with web pages programmatically, 
# as if a user were clicking on buttons or links.
time.sleep(5)
presidencial = driver.find_element(By.XPATH, "/html/body/onpe-root/onpe-layout-container/onpe-menu/div/nav/div/div/div[2]/div/div[2]/a")
presidencial.click()

In [5]:
#  these lines of code automate the process of selecting an option from a dropdown menu on the webpage. 
# The first part clicks on the dropdown to open it, and the second part selects a specific option within the dropdown (in this case, the second option). 
# This is a common approach in web automation to interact with dropdown menus programmatically.

ambito = driver.find_element(By.ID, "select_ambito")

ambito.click()

peru_opt = driver.find_element(By.XPATH, "/html/body/onpe-root/onpe-layout-container/onpe-onpe-epres-re/div[1]/div[3]/div[1]/div[1]/div/div/div/select/option[2]")

peru_opt.click()

In [6]:
# Creating empty lists that will later contain the information of the departments, provinces, districts, etc.
list_departments = []
list_provinces = []
list_districts = []
list_tablas = []

# Creating a loop using the "for" function to obtain, iteratively, the index of each department.
# This is because all department paths are differentiated only by the last number (department index)
# Using ".text" to extract the visible text of an element on a web page.
# Using ".append", we add the obtained information to the list "list_departments".
# We repeat the same procedure in the case of "provinces" and "districts"

time.sleep(5)

for idx_dpto in range(2, 10000000):
    try:
        dpto_opt = driver.find_element(By.XPATH, f'/html/body/onpe-root/onpe-layout-container/onpe-onpe-epres-re/div[1]/div[3]/div[1]/div[1]/div/div/div[2]/select/option[{idx_dpto}]')
        dpto_name = dpto_opt.text
        dpto_opt.click()
        list_departments.append(dpto_name)
        
        
        for idx_prov in range(2, 10000000):
            try:
                prov_opt = driver.find_element(By.XPATH, f'/html/body/onpe-root/onpe-layout-container/onpe-onpe-epres-re/div[1]/div[3]/div[1]/div[1]/div/div/div[3]/select/option[{idx_prov}]')
                prov_name = prov_opt.text
                prov_opt.click()
                list_provinces.append(prov_name)
                   
                for idx_dist in range(2,10000000):
                    try:
                        dist_opt = driver.find_element(By.XPATH, f"/html/body/onpe-root/onpe-layout-container/onpe-onpe-epres-re/div[1]/div[3]/div[1]/div[1]/div/div/div[4]/select/option[{idx_dist}]")
                        dist_name = dist_opt.text
                        dist_opt.click()
                        list_districts.append(dist_name)
        
           
                        # To extract the table, we use ".find_element" by ID.
                        # Then to get the table in HTML we use ".get_property"
                        # We obtain the clean vote table by reading it with PANDAS using "pd.read_html"
                        tabla_votos = driver.find_element(By.ID, "table-scroll")
                        tabla_html = tabla_votos.get_property('innerHTML')
                        tabla_votos_clean = pd.read_html(tabla_html)[0]
                        
                        #Since the first two columns have 'Nan' values, it's better if we drop them, since they dont bring new info to the table. Also, we dont wanna know the values of %VALIDOS and %EMITIDOS, only the total votes,
                        #so it's also better if we drop them.
                        columns_to_drop = [0, 1, 4, 5]
                        tabla_votos_clean = tabla_votos_clean.drop(tabla_votos_clean.columns[columns_to_drop], axis=1)
                        
                        # Change the names of columns
                        new_column_names = ['political_organization', 'total_votes']  
                        tabla_votos_clean.columns = new_column_names
                       
                        # Delete the first row of the DataFrame
                        tabla_votos_clean = tabla_votos_clean.iloc[1:]
                        
                        # Assign values to the corresponding columns
                        tabla_votos_clean['department'] = dpto_name
                        tabla_votos_clean['province'] = prov_name
                        tabla_votos_clean['district'] = dist_name

                        # Now that we have the DataFrame, we add it to the previously created list of tables "list_tablas"
                        list_tablas.append(tabla_votos_clean)
                    
                    except:
                        break
            except:
                break
    except:
        break
           
# We use the "pd.concat()" function to concatenate (join) DataFrames along a specific axis            
final_table = pd.concat(list_tablas, ignore_index=True)


final_table 

,political_organization,total_votes,department,province,district
0,PARTIDO NACIONALISTA PERUANO,356,AMAZONAS,BAGUA,BAGUA
1,"EL FRENTE AMPLIO POR JUSTICIA, VIDA Y LIBERTAD",13,AMAZONAS,BAGUA,BAGUA
2,PARTIDO MORADO,41,AMAZONAS,BAGUA,BAGUA
3,PERU PATRIA SEGURA,4,AMAZONAS,BAGUA,BAGUA
4,VICTORIA NACIONAL,151,AMAZONAS,BAGUA,BAGUA
...,...,...,...,...,...
20191,ALIANZA PARA EL PROGRESO,292,UCAYALI,PURUS,PURUS
20192,TOTAL DE VOTOS VÁLIDOS,825,UCAYALI,PURUS,PURUS
20193,VOTOS EN BLANCO,422,UCAYALI,PURUS,PURUS
20194,VOTOS NULOS,182,UCAYALI,PURUS,PURUS
